##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# 회귀: 주택 가격 예측하기

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/basic_regression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />TensorFlow.org에서 보기</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/ko/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩(Colab)에서 실행하기</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/ko/tutorials/keras/basic_regression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />깃허브(GitHub) 소스 보기</a>
  </td>
</table>

*회귀*(regression) 문제는 가격이나 확률 같이 연속된 출력 값을 예측하는 것이 목적입니다. 이와는 달리 *분류*(classification) 문제는 이산적인 레이블(label)을 예측하는 것이 목적입니다(예를 들어, 사과 이미지 또는 오렌지 이미지).

이 노트북은 1970년대 중반 보스턴 교외 주택의 중간 가격을 예측하는 모델을 만듭니다. 이를 위해 범죄율과 재산세율 같은 이 지역에 관한 데이터를 모델에 제공하겠습니다.

이 예제는 `tf.keras` API를 사용합니다. 자세한 내용은 [케라스 가이드](https://www.tensorflow.org/guide/keras)를 참고하세요.

In [0]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
from tensorflow import keras

import numpy as np

print(tf.__version__)

## 보스턴 주택 가격 데이터셋

이 [데이터셋](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)은 텐서플로에서 바로 가져올 수 있습니다. 다운로드한 후에 훈련 세트를 섞겠습니다:

In [0]:
boston_housing = keras.datasets.boston_housing

(train_data, train_labels), (test_data, test_labels) = boston_housing.load_data()

# 훈련 세트를 섞습니다
order = np.argsort(np.random.random(train_labels.shape))
train_data = train_data[order]
train_labels = train_labels[order]

### 샘플과 특성

이 데이터셋은 이전에 사용했던 것보다 훨씬 작습니다. 전체 샘플 506개 중에서 404개는 훈련 세트로 102개는 테스트 세트로 나뉘어져 있습니다:

In [0]:
print("훈련 세트: {}".format(train_data.shape))  # 404개 샘플, 13개 특성
print("테스트 세트:  {}".format(test_data.shape))   # 102개 샘플, 13개 특성

이 데이터셋은 13개의 특성을 가집니다:

1. 인당 범죄율
2. 25,000 평방 피트가 넘는 주택 비율
3. 도시당 소매 업종이 아닌 지역 비율
4. 찰스강 인접 여부(강 주변=1, 그외=0)
5. 일산화질소 농도(10 ppm 당)
6. 주택의 평균 방 개수
7. 1940년 이전에 지어진 자가 소유 주택 비율
8. 다섯 개의 보스턴 고용 센터까지 가중치가 적용된 거리
9. 방사형으로 뻗은 고속도로까지 접근성 지수
10. $10,000당 재산세율
11. 도시의 학생-교사 비율
12. 1000(Bk - 0.63)^2, 여기에서 Bk는 도시의 아프리카계 미국인 비율
13. 저소득 계층의 비율

입력 특성들은 각기 다른 스케일로 저장되어 있습니다. 0과 1 사이의 비율로 표현되어 있는 특성도 있고, 1과 12 사이의 범위를 가진 특성도 있고, 0과 100 사이의 범위를 가진 특성도 있습니다. 이런 경우는 실제로 흔합니다. 이런 데이터를 탐색하고 정제하는 기술을 익히는 것이 중요합니다.

중요 포인트: 모델러(modeler)와 개발자 입장에서 이 데이터를 어떻게 사용할지, 모델의 예측이 만들 수 있는 장점과 단점을 생각해 보세요. 이런 모델은 사회적 편견과 차이를 부추킬 수 있습니다. 해결하고자 하는 문제와 관련된 특성인가요? 아니면 편견을 만들게 되나요? 더 자세한 정보는 [공정한 머신러닝](https://developers.google.com/machine-learning/fairness-overview/)을 참고하세요.

In [0]:
print(train_data[0])  # 샘플 하나를 출력하여 특성의 스케일을 확인합니다

[판다스](https://pandas.pydata.org) 라이브러리를 사용하여 데이터셋에 있는 처음 몇 개의 행을 미려하게 출력해 보죠:

In [0]:
import pandas as pd

column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD',
                'TAX', 'PTRATIO', 'B', 'LSTAT']

df = pd.DataFrame(train_data, columns=column_names)
df.head()

### 레이블

레이블은 천 단위 달러의 주택 가격입니다. (1970년대 중반의 가격입니다.)

In [0]:
print(train_labels[0:10])  # 처음 10개의 값을 출력합니다

## 특성 정규화

스케일과 범위가 다른 특성은 정규화(normalization)하는 것이 권장됩니다. 각 특성별로 특성의 평균을 빼고 표준 편차로 나눕니다:

In [0]:
# mean과 std를 계산할 때 테스트 데이터는 *사용하지 않습니다*

mean = train_data.mean(axis=0)
std = train_data.std(axis=0)
train_data = (train_data - mean) / std
test_data = (test_data - mean) / std

print(train_data[0])  # 정규화된 첫 번째 샘플

특성을 정규화하지 않더라도 모델이 *수렴할 수 있지만*, 훈련하기 어렵고 입력의 단위에 의존적인 모델이 만들어집니다.

## 모델 생성

모델을 구성해 보죠. 여기에서는 두 개의 완전 연결(densely connected) 은닉층으로 `Sequential` 모델을 만들겠습니다. 출력 층은 하나의 연속적인 값을 반환합니다. 나중에 두 번째 모델을 만들기 위해 `build_model` 함수로 모델 구성 단계를 감쌉니다.

In [0]:
def build_model():
  model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu,
                       input_shape=(train_data.shape[1],)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(1)
  ])

  optimizer = tf.train.RMSPropOptimizer(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae'])
  return model

model = build_model()
model.summary()

## 모델 훈련

이 모델을 500번의 에포크 동안 훈련합니다. 훈련 정확도와 검증 정확도는 `history` 객체에 기록됩니다.

In [0]:
# 에포크가 끝날 때마다 점(.)을 출력해 훈련 진행 과정을 표시합니다
class PrintDot(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0: print('')
    print('.', end='')

EPOCHS = 500

# 훈련 통계를 저장합니다
history = model.fit(train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[PrintDot()])

`history` 객체에 저장된 통계치를 사용해 모델의 훈련 과정을 시각화해 보죠. 이 데이터를 사용해 모델 성능이 최대가 되려면 얼마나 걸리는지 확인하겠습니다.

In [0]:
import matplotlib.pyplot as plt


def plot_history(history):
  plt.figure()
  plt.xlabel('Epoch')
  plt.ylabel('Mean Abs Error [1000$]')
  plt.plot(history.epoch, np.array(history.history['mean_absolute_error']),
           label='Train Loss')
  plt.plot(history.epoch, np.array(history.history['val_mean_absolute_error']),
           label = 'Val loss')
  plt.legend()
  plt.ylim([0, 5])

plot_history(history)

이 그래프는 200번째 에포크 이후에 모델이 거의 향상되지 않는 것 같습니다. `model.fit` 메서드를 수정하여 검증 점수가 향상되지 않으면 자동으로 훈련을 멈추도록 만들어 보죠. 에포크마다 훈련 상태를 점검하기 위해 *콜백*(callback) 함수를 사용하겠습니다. 지정된 에포크 횟수 동안 성능 향상이 없으면 자동으로 훈련이 멈춥니다.

이 콜백에 대해 더 자세한 내용은 [여기](https://www.tensorflow.org/versions/master/api_docs/python/tf/keras/callbacks/EarlyStopping)를 참고하세요.

In [0]:
model = build_model()

# patience 매개변수는 성능 향상을 체크할 에포크 횟수입니다
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(train_data, train_labels, epochs=EPOCHS,
                    validation_split=0.2, verbose=0,
                    callbacks=[early_stop, PrintDot()])

plot_history(history)

이 그래프를 보면 평균 절대 오차가 약 2,500 달러입니다. 좋은 결과인가요? 어떤 주택 가격은 15,000 달러에 불과하기 때문에 2,500 달러는 작지 않은 크기입니다.

테스트 세트에서 모델의 성능을 확인해 보죠:

In [0]:
[loss, mae] = model.evaluate(test_data, test_labels, verbose=0)

print("테스트 세트의 평균 절대 오차: ${:7.2f}".format(mae * 1000))

## 예측

마지막으로 테스트 세트에 있는 샘플을 사용해 주택 가격을 예측해 보겠습니다:

In [0]:
test_predictions = model.predict(test_data).flatten()

plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [1000$]')
plt.ylabel('Predictions [1000$]')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
_ = plt.plot([-100, 100], [-100, 100])


In [0]:
error = test_predictions - test_labels
plt.hist(error, bins = 50)
plt.xlabel("Prediction Error [1000$]")
_ = plt.ylabel("Count")

## 결론

이 노트북은 회귀 문제를 위한 기법을 소개합니다.

* 평균 제곱 오차(MSE)는 회귀 문제에서 자주 사용하는 손실 함수입니다(분류 문제에서 사용하는 것과는 다릅니다).
* 비슷하게 회귀에서 사용되는 평가 지표도 분류와 다릅니다. 많이 사용하는 회귀 지표는 평균 절댓값 오차(MAE)입니다.
* 입력 데이터의 특성이 여러 가지 범위를 가질 때 각 특성의 스케일을 독립적으로 조정해야 합니다.
* 훈련 데이터가 많지 않다면 과대적합을 피하기 위해 은닉층의 개수가 적은 소규모 네트워크가 좋습니다.
* 조기 종료(Early stopping)은 과대적합을 방지하기 위한 좋은 방법입니다.